In [ ]:
# ============================================================
# AV-Predict
# Authors: Danieli Soares de Oliveira and Clainer Bravin Donadel
# License: MIT
# ============================================================

import os
import io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt  # <- for plots

from IPython.display import display, Markdown
from google.colab import files

import ipywidgets as widgets

from sklearn.model_selection import KFold, GroupKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

# -------------------------------
# 1) Utility: Safe file loading
# -------------------------------
def load_excel_or_ask(default_names=("Dataset.xlsx", "/content/Dataset.xlsx", "/mnt/data/Dataset.xlsx")):
    """
    Try loading Dataset.xlsx from common locations, else ask the user to upload.
    (No dataset preview is shown to avoid confusion.)
    """
    for p in default_names:
        if os.path.exists(p):
            try:
                df = pd.read_excel(p)
                print(f"Loaded dataset from: {p}")
                return df
            except Exception as e:
                print(f"Found but failed to read {p}: {e}")

    print("Dataset.xlsx not found in default locations. Please upload it now.")
    uploaded = files.upload()
    for fname in uploaded.keys():
        if fname.lower().endswith((".xlsx", ".xls")):
            df = pd.read_excel(io.BytesIO(uploaded[fname]))
            print(f"Loaded dataset from uploaded file: {fname}")
            return df

    raise FileNotFoundError("No Excel file uploaded. Please run again and upload Dataset.xlsx.")


def try_show_readme(default_names=("README.txt", "/content/README.txt", "/mnt/data/README.txt")):
    """
    If README.txt exists, display it to the user for quick reference (optional).
    """
    for p in default_names:
        if os.path.exists(p):
            try:
                with open(p, "r", encoding="utf-8") as f:
                    text = f.read()
                display(Markdown("#### README.txt (column glossary)"))
                print(text)
                return
            except Exception as e:
                print(f"Found README at {p} but failed to read: {e}")
                return
    # Silent if not found.


# -----------------------------------------
# 2) Column discovery & normalization
# -----------------------------------------
def normalize_columns(df):
    """
    Standardize column names to canonical names from the README.
    """
    variants = {
        "coagulant concentration": "Coagulant concentration [mL/L]",
        "coagulation/flocculation time": "Coagulation/flocculation time [min]",
        "coagulation time": "Coagulation/flocculation time [min]",
        "flocculation time": "Coagulation/flocculation time [min]",
        "mixing speed": "Mixing speed [rpm]",
        "global velocity gradient": "Global velocity gradient [s-1]",
        "sedimentation time": "Sedimentation time [min]",
        "turbidity": "Turbidity [NTU]",
        "trior": "TRIOR",
        "trial": "TRIOR",
        "replicate": "TRIOR",
        "replication": "TRIOR",
        # Possible explicit initial turbidity column (kept for completeness)
        "initial turbidity": "Turbidity at time zero [NTU]",
        "turbidity at time zero": "Turbidity at time zero [NTU]",
        "t0": "Turbidity at time zero [NTU]",
        "turbidity_0": "Turbidity at time zero [NTU]",
        "turbidez inicial": "Turbidity at time zero [NTU]",
    }

    def fp(s):
        s = str(s).lower()
        for ch in "[](){}":
            s = s.replace(ch, "")
        s = s.replace("  ", " ")
        return s.strip()

    mapping = {}
    for col in df.columns:
        fcol = fp(col)
        target = None
        for key, can in variants.items():
            if key in fcol:
                target = can
                break
        mapping[col] = target if target else col

    return df.rename(columns=mapping), mapping


# -----------------------------------------
# 3) Time-zero index & helpers
# -----------------------------------------
def build_timezero_index(df, timezero_feature_candidates, target_col):
    """
    Builds a nearest-neighbor index using only time-zero rows to later
    (a) compute the training feature 'Turbidity at time zero [NTU]' per row.
    """
    if "Sedimentation time [min]" not in df.columns:
        return None

    mask = (df["Sedimentation time [min]"] == 0)
    tz_df = df.loc[mask, timezero_feature_candidates + [target_col]].dropna()
    if len(tz_df) == 0:
        return None

    TZ_FEATS = tz_df[timezero_feature_candidates].astype(float).values
    TZ_TARGET = tz_df[target_col].astype(float).values
    tz_mean = TZ_FEATS.mean(axis=0)
    tz_std = TZ_FEATS.std(axis=0, ddof=0)
    tz_std[tz_std == 0.0] = 1.0
    TZ_FEATS_STD = (TZ_FEATS - tz_mean) / tz_std

    index = {
        "columns": list(timezero_feature_candidates),
        "mean": tz_mean,
        "std": tz_std,
        "X_std": TZ_FEATS_STD,
        "y": TZ_TARGET,
        "k": min(5, len(tz_df)),
    }
    return index


def estimate_T0_from_timezero_index(timezero_index, query_dict):
    """
    Estimate T0 from nearest neighbors among time-zero records.
    """
    if timezero_index is None:
        return None

    cols = timezero_index["columns"]
    mean = timezero_index["mean"]
    std = timezero_index["std"]
    X_std = timezero_index["X_std"]
    y = timezero_index["y"]
    k = timezero_index["k"]

    q = np.array([float(query_dict[c]) for c in cols], dtype=float)
    q_std = (q - mean) / std
    d = np.linalg.norm(X_std - q_std, axis=1)

    tol = 1e-9
    exact = (d <= tol)
    if np.any(exact):
        return float(np.mean(y[exact]))

    idx = np.argpartition(d, k-1)[:k]
    d_k = d[idx]
    y_k = y[idx]
    w = 1.0 / (d_k + 1e-12)
    w = w / w.sum()
    return float(np.sum(w * y_k))


def add_T0_feature_for_training(df, feature_cols, timezero_index, target_col):
    """
    Create the feature 'Turbidity at time zero [NTU]' for every row.
    - If Sedimentation time is 0: use the row's target as T0 (identity).
    - Else: estimate from time-zero index using similar conditions.
    If everything fails, fallback to 95th percentile of target.
    Returns: (t0_name, T0_feat np.array)
    """
    t0_name = "Turbidity at time zero [NTU]"
    T0_feat = np.zeros(len(df), dtype=float)
    fallback = float(df[target_col].quantile(0.95)) if len(df) else 1.0

    has_sed = "Sedimentation time [min]" in df.columns
    for i, row in df.iterrows():
        if has_sed and row["Sedimentation time [min]"] == 0 and pd.notnull(row[target_col]):
            T0_feat[i] = float(row[target_col])
        else:
            if timezero_index is not None:
                query = {}
                ok = True
                for c in timezero_index["columns"]:
                    if c in df.columns and pd.notnull(row[c]):
                        query[c] = float(row[c])
                    else:
                        ok = False
                        break
                if ok:
                    est = estimate_T0_from_timezero_index(timezero_index, query)
                    if est is not None and np.isfinite(est) and est > 0:
                        T0_feat[i] = float(est)
                        continue
            T0_feat[i] = fallback

    return t0_name, T0_feat


# -----------------------------------------
# 4) Data cleaning and grouping
# -----------------------------------------
def clean_and_prepare(df):
    """
    Build features/targets, groups for GroupKFold, and add the new feature:
    'Turbidity at time zero [NTU]' for training (from dataset).
    Targets returned:
      - y_turb: final turbidity [NTU]
      - y_eff:  removal efficiency [%] computed from dataset T0 (clipped 0..100)
    """
    feature_candidates = [
        "Coagulant concentration [mL/L]",
        "Coagulation/flocculation time [min]",
        "Mixing speed [rpm]",
        "Global velocity gradient [s-1]",
        "Sedimentation time [min]",
    ]
    timezero_feature_candidates = [
        "Coagulant concentration [mL/L]",
        "Coagulation/flocculation time [min]",
        "Mixing speed [rpm]",
        "Global velocity gradient [s-1]",
    ]
    target_col = "Turbidity [NTU]"

    # Keep only relevant columns if present
    keep_cols = [c for c in feature_candidates if c in df.columns]
    if target_col in df.columns:
        keep_cols.append(target_col)
    else:
        raise ValueError("Target column 'Turbidity [NTU]' not found in dataset.")
    df = df[keep_cols].copy()

    # Coerce numeric and drop missing
    for c in keep_cols:
        df[c] = pd.to_numeric(df[c], errors='coerce')
    df = df.dropna()
    df = df[df[target_col] >= 0].reset_index(drop=True)

    # Build time-zero index from dataset
    tz_index = build_timezero_index(df, timezero_feature_candidates, target_col)

    # Add the new training feature (T0)
    t0_name, T0_feat = add_T0_feature_for_training(df, feature_candidates, tz_index, target_col)
    df[t0_name] = T0_feat

    # Compute efficiency [%] from dataset using T0_feat (clip 0..100)
    eps = 1e-12
    valid_t0 = np.maximum(df[t0_name].values.astype(float), eps)
    y_turb = df[target_col].values.astype(float)
    y_eff = 100.0 * (valid_t0 - y_turb) / valid_t0
    y_eff = np.clip(y_eff, 0.0, 100.0)

    # Final feature set for training (includes T0)
    feature_cols = [c for c in feature_candidates if c in df.columns] + [t0_name]

    # X
    X = df[feature_cols].copy()

    # Group identical experimental settings (excluding target);
    # use the original 5 process features to group (not including T0)
    def make_group_id(row, decimals=6):
        cols = [c for c in feature_candidates if c in df.columns]
        tup = tuple(np.round(row[cols].values.astype(float), decimals))
        return "|".join(map(str, tup))

    group_series = X.apply(make_group_id, axis=1)
    groups, _ = pd.factorize(group_series, sort=True)

    return X, y_turb, y_eff, valid_t0, groups, feature_cols, t0_name


# -----------------------------------------
# 5) Modeling utilities and metrics
# -----------------------------------------
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))


def build_models(random_state=42):
    """
    Create base pipelines for regressors that will be trained to predict Efficiency [%].
    """
    models = {}
    models["Linear Regression"] = Pipeline(steps=[
        ("model", LinearRegression())
    ])
    models["K-Nearest Neighbors Regressor"] = Pipeline(steps=[
        ("scaler", StandardScaler()),
        ("model", KNeighborsRegressor(n_neighbors=5))
    ])
    models["Support Vector Regressor with Radial Basis Function kernel"] = Pipeline(steps=[
        ("scaler", StandardScaler()),
        ("model", SVR(C=10.0, epsilon=0.1, kernel="rbf"))
    ])
    models["Random Forest Regressor"] = Pipeline(steps=[
        ("model", RandomForestRegressor(
            n_estimators=300, max_depth=None, random_state=random_state, n_jobs=-1
        ))
    ])
    models["Gradient Boosting Regressor"] = Pipeline(steps=[
        ("model", GradientBoostingRegressor(
            random_state=random_state, learning_rate=0.05, n_estimators=500, max_depth=3
        ))
    ])
    models["Artificial Neural Network (Multilayer Perceptron Regressor)"] = Pipeline(steps=[
        ("scaler", StandardScaler()),
        ("model", MLPRegressor(
            hidden_layer_sizes=(64, 32),
            activation="relu",
            solver="adam",
            learning_rate_init=0.001,
            alpha=0.0001,            # L2 penalty
            max_iter=300,
            early_stopping=True,
            validation_fraction=0.1,
            random_state=random_state
        ))
    ])
    return models


def evaluate_with_groupkfold(pipe, X, y_turb, y_eff, t0_vec, groups, n_splits=5, return_oof=False):
    """
    Manual GroupKFold that trains on Efficiency[%] and evaluates in terms of final turbidity.
    If return_oof=True, also returns out-of-fold arrays for plotting (efficiency & turbidity).
    """
    if groups is not None:
        cv = GroupKFold(n_splits=n_splits)
        splits = cv.split(X, y_eff, groups=groups)
    else:
        cv = KFold(n_splits=n_splits, shuffle=True, random_state=42)
        splits = cv.split(X, y_eff)

    r2_list, rmse_list = [], []
    if return_oof:
        oof_eff_true, oof_eff_pred = np.zeros(len(y_eff)), np.zeros(len(y_eff))
        oof_turb_true, oof_turb_pred = np.zeros(len(y_turb)), np.zeros(len(y_turb))

    for train_idx, valid_idx in splits:
        X_tr, X_va = X.iloc[train_idx], X.iloc[valid_idx]
        y_eff_tr, y_turb_va = y_eff[train_idx], y_turb[valid_idx]
        t0_va = t0_vec[valid_idx]

        pipe.fit(X_tr, y_eff_tr)
        eff_pred = pipe.predict(X_va)
        eff_pred = np.clip(eff_pred, 0.0, 100.0)

        y_pred_turb = t0_va * (1.0 - eff_pred / 100.0)
        r2_list.append(r2_score(y_turb_va, y_pred_turb))
        rmse_list.append(rmse(y_turb_va, y_pred_turb))

        if return_oof:
            oof_eff_true[valid_idx] = y_eff[valid_idx]
            oof_eff_pred[valid_idx] = eff_pred
            oof_turb_true[valid_idx] = y_turb_va
            oof_turb_pred[valid_idx] = y_pred_turb

    y_mean = float(np.mean(y_turb)) if len(y_turb) else np.nan
    rmse_arr = np.array(rmse_list, dtype=float)
    r2_arr = np.array(r2_list, dtype=float)
    nrmse_pct = 100.0 * (rmse_arr / y_mean) if y_mean and np.isfinite(y_mean) and y_mean != 0 else np.full_like(rmse_arr, np.nan)

    results = {
        "R2_mean": float(np.mean(r2_arr)),
        "R2_std": float(np.std(r2_arr)),
        "RMSE_mean": float(np.mean(rmse_arr)),
        "RMSE_std": float(np.std(rmse_arr)),
        "NRMSE_pct_mean": float(np.mean(nrmse_pct)),
        "NRMSE_pct_std": float(np.std(nrmse_pct)),
        "y_mean": y_mean
    }
    if return_oof:
        results["oof"] = {
            "eff_true": oof_eff_true,
            "eff_pred": oof_eff_pred,
            "turb_true": oof_turb_true,
            "turb_pred": oof_turb_pred
        }
    return results


# -----------------------------------------
# 6) Load & prepare dataset (no preview)
# -----------------------------------------
raw_df = load_excel_or_ask()
df_norm, col_map = normalize_columns(raw_df)
try_show_readme()

(
    X,           # features incl. T0 (derived from dataset)
    Y_TURB,      # final turbidity [NTU]
    Y_EFF,       # efficiency [%] from dataset (clipped 0..100)
    T0_VEC,      # derived T0 per row
    GROUPS,      # grouping for CV
    FEATURE_COLUMNS,
    T0_FEATURE_NAME
) = clean_and_prepare(df_norm)

# -----------------------------------------
# 7) Build models
# -----------------------------------------
models_dict = build_models(random_state=42)


# -----------------------------------------
# 8) Interactive UI (ipywidgets) with 3 sections
# -----------------------------------------
FULL_LABEL_STYLE = {'description_width': 'initial'}
WIDE = widgets.Layout(width='680px')  # keep hyperparameter controls wide

CARD_LAYOUT = widgets.Layout(
    border='1px solid #ddd',
    padding='16px',
    margin='12px 0',
    width='100%',
    overflow_x='visible',
    overflow_y='visible'
)
TITLE_LAYOUT = widgets.Layout(margin='0 0 12px 0')

# ---- Model selector (DEFAULT: Linear Regression)
model_dropdown = widgets.Dropdown(
    options=list(models_dict.keys()),
    value="Linear Regression",
    description="Model:",
    layout=WIDE,
    style=FULL_LABEL_STYLE
)

# ---- Hyperparameters (full labels + meanings)
# KNN
knn_k = widgets.IntSlider(
    value=5, min=1, max=50, step=1,
    description="Number of neighbors:",
    layout=WIDE, style=FULL_LABEL_STYLE
)
knn_k_note = widgets.HTML(
    "<small><b>Number of neighbors</b>: count of nearest data points used to make the prediction. "
    "Smaller values capture local patterns; larger values smooth predictions.</small>"
)

# SVR
svr_c = widgets.FloatLogSlider(
    value=10.0, base=10, min=-2, max=2, step=0.1,
    description="Regularization parameter:",
    layout=WIDE, style=FULL_LABEL_STYLE
)
svr_c_note = widgets.HTML(
    "<small><b>Regularization parameter</b>: controls the penalty for errors. "
    "Higher values fit the training data more closely; lower values increase regularization, creating a simpler model.</small>"
)
svr_eps = widgets.FloatLogSlider(
    value=0.1, base=10, min=-3, max=1, step=0.1,
    description="Epsilon parameter for the epsilon-insensitive loss:",
    layout=WIDE, style=FULL_LABEL_STYLE
)
svr_eps_note = widgets.HTML(
    "<small><b>Epsilon parameter for the epsilon-insensitive loss</b>: defines a no-penalty zone around predictions. "
    "Errors within this zone are ignored; larger values yield a smoother, less sensitive model.</small>"
)

# Random Forest
rf_n = widgets.IntSlider(
    value=300, min=50, max=1000, step=50,
    description="Number of trees (estimators):",
    layout=WIDE, style=FULL_LABEL_STYLE
)
rf_n_note = widgets.HTML(
    "<small><b>Number of trees (estimators)</b>: more trees can reduce variance but increase training time.</small>"
)
rf_depth = widgets.IntSlider(
    value=0, min=0, max=30, step=1,
    description="Maximum tree depth (0 means unlimited):",
    layout=WIDE, style=FULL_LABEL_STYLE
)
rf_depth_note = widgets.HTML(
    "<small><b>Maximum tree depth</b>: zero means unlimited depth. "
    "Shallower trees may generalize better; very deep trees can overfit.</small>"
)

# Gradient Boosting
gb_n = widgets.IntSlider(
    value=500, min=100, max=1500, step=100,
    description="Number of boosting stages (estimators):",
    layout=WIDE, style=FULL_LABEL_STYLE
)
gb_n_note = widgets.HTML(
    "<small><b>Number of boosting stages (estimators)</b>: number of weak learners added sequentially.</small>")
gb_lr = widgets.FloatLogSlider(
    value=0.05, base=10, min=-3, max=0, step=0.05,
    description="Learning rate:",
    layout=WIDE, style=FULL_LABEL_STYLE
)
gb_lr_note = widgets.HTML(
    "<small><b>Learning rate</b>: contribution of each weak learner. "
    "Lower values often require more stages and may generalize better.</small>")
gb_depth = widgets.IntSlider(
    value=3, min=1, max=8, step=1,
    description="Maximum tree depth:",
    layout=WIDE, style=FULL_LABEL_STYLE
)
gb_depth_note = widgets.HTML(
    "<small><b>Maximum tree depth</b>: depth of the individual weak learners and strength of feature interactions.</small>"
)

# ANN (MLPRegressor)
ann_layers = widgets.IntSlider(
    value=2, min=1, max=4, step=1,
    description="Number of hidden layers:",
    layout=WIDE, style=FULL_LABEL_STYLE
)
ann_layers_note = widgets.HTML(
    "<small><b>Number of hidden layers</b>: depth of the neural network. "
    "More layers can model more complex relationships but may require more data.</small>"
)
ann_h1 = widgets.IntSlider(
    value=64, min=4, max=512, step=4,
    description="Number of neurons in hidden layer 1:",
    layout=WIDE, style=FULL_LABEL_STYLE
)
ann_h2 = widgets.IntSlider(
    value=32, min=4, max=512, step=4,
    description="Number of neurons in hidden layer 2:",
    layout=WIDE, style=FULL_LABEL_STYLE
)
ann_h3 = widgets.IntSlider(
    value=16, min=4, max=512, step=4,
    description="Number of neurons in hidden layer 3:",
    layout=WIDE, style=FULL_LABEL_STYLE
)
ann_h4 = widgets.IntSlider(
    value=8, min=4, max=512, step=4,
    description="Number of neurons in hidden layer 4:",
    layout=WIDE, style=FULL_LABEL_STYLE
)
ann_act = widgets.Dropdown(
    options=[
        ("Rectified Linear Unit (ReLU)", "relu"),
        ("Hyperbolic tangent (tanh)", "tanh"),
        ("Logistic sigmoid (logistic)", "logistic"),
        ("Identity (linear)", "identity"),
    ],
    value="relu",
    description="Activation function:",
    layout=WIDE, style=FULL_LABEL_STYLE
)
ann_act_note = widgets.HTML(
    "<small><b>Activation function</b>: non-linear transformation applied in each neuron; "
    "it defines how complex patterns are modeled.</small>"
)
ann_solver = widgets.Dropdown(
    options=[
        ("Adam optimizer", "adam"),
        ("Limited-memory BFGS (lbfgs)", "lbfgs"),
        ("Stochastic gradient descent (sgd)", "sgd"),
    ],
    value="adam",
    description="Optimization algorithm (solver):",
    layout=WIDE, style=FULL_LABEL_STYLE
)
ann_solver_note = widgets.HTML(
    "<small><b>Optimization algorithm (solver)</b>: method used to minimize the loss function during training.</small>"
)
ann_lr_init = widgets.FloatLogSlider(
    value=1e-3, base=10, min=-5, max=-1, step=0.1,
    description="Initial learning rate:",
    layout=WIDE, style=FULL_LABEL_STYLE
)
ann_lr_init_note = widgets.HTML(
    "<small><b>Initial learning rate</b>: step size of the optimizer; too large may diverge, too small may be slow.</small>"
)
ann_alpha = widgets.FloatLogSlider(
    value=1e-4, base=10, min=-7, max=-1, step=0.1,
    description="L2 regularization strength (alpha):",
    layout=WIDE, style=FULL_LABEL_STYLE
)
ann_alpha_note = widgets.HTML(
    "<small><b>L2 regularization strength (alpha)</b>: penalty for large weights; helps prevent overfitting.</small>"
)
ann_max_iter = widgets.IntSlider(
    value=300, min=100, max=2000, step=50,
    description="Maximum number of training iterations:",
    layout=WIDE, style=FULL_LABEL_STYLE
)
ann_max_iter_note = widgets.HTML(
    "<small><b>Maximum number of training iterations</b>: upper bound on training steps; "
    "increase if the model does not converge.</small>"
)
ann_early = widgets.Checkbox(
    value=True,
    description="Enable early stopping during training (with an internal validation split)",
    indent=False,
    layout=WIDE
)
ann_early_note = widgets.HTML(
    "<small><b>Early stopping</b>: stops training when validation score does not improve, helping to avoid overfitting.</small>"
)

# ---- Inputs (full names). Three integer-only fields. Fields are 214px wide.
def_median = X.median(numeric_only=True)

def get_or_default(col, default):
    try:
        return float(def_median.get(col, default))
    except Exception:
        return float(default)

def make_param_block_float(title_html, default_value, note_html):
    title = widgets.HTML(f"<b>{title_html}</b>", layout=TITLE_LAYOUT)
    field = widgets.FloatText(value=float(default_value), layout=widgets.Layout(width="214px"))
    note = widgets.HTML(note_html)
    return widgets.VBox([title, field, note], layout=widgets.Layout(width='100%'))

def make_param_block_int(title_html, default_value, note_html):
    title = widgets.HTML(f"<b>{title_html}</b>", layout=TITLE_LAYOUT)
    field = widgets.IntText(value=int(round(float(default_value))), layout=widgets.Layout(width="214px"))
    note = widgets.HTML(note_html)
    return widgets.VBox([title, field, note], layout=widgets.Layout(width='100%'))

blk_t0 = make_param_block_float(
    "Turbidity at time zero [NTU]",
    100.0,  # Default as requested
    "<small>Reference turbidity measured at time zero (before sedimentation). "
    "Used directly by the model (feature) and for efficiency/turbidity calculation.</small>"
)
blk_conc = make_param_block_float(
    "Coagulant concentration [mL/L]",
    get_or_default("Coagulant concentration [mL/L]", 0.0),
    "<small>Dosage of Aloe vera coagulant used (milliliters per liter of water).</small>"
)
blk_ctime = make_param_block_float(
    "Coagulation/flocculation time [min]",
    get_or_default("Coagulation/flocculation time [min]", 0.0),
    "<small>Total time for coagulation and flocculation phases (minutes).</small>"
)
blk_mix = make_param_block_int(
    "Mixing speed [rpm]",
    get_or_default("Mixing speed [rpm]", 0.0),
    "<small>Impeller or agitator rotational speed during mixing (revolutions per minute). Integer only.</small>"
)
blk_g = make_param_block_int(
    "Global velocity gradient [s-1]",
    get_or_default("Global velocity gradient [s-1]", 0.0),
    "<small>Global velocity gradient, a measure of shear intensity (per second). Integer only.</small>"
)
blk_sed = make_param_block_int(
    "Sedimentation time [min]",
    get_or_default("Sedimentation time [min]", 0.0),
    "<small>Settling time after mixing, before sampling turbidity (minutes). Integer only.</small>"
)

# ---- Buttons & output (unchanged widths: 578px)
btn_layout = widgets.Layout(width='578px', height='auto')
train_button = widgets.Button(
    description="Train and Evaluate (Group 5-fold Cross-Validation)",
    button_style="primary",
    layout=btn_layout
)
predict_button = widgets.Button(
    description="Predict Turbidity [NTU] and Turbidity Removal Efficiency [%]",
    button_style="success",
    layout=btn_layout
)
output_area = widgets.Output()

# -----------------------------------------
# 9) Wiring (callbacks and pipeline rebuild)
# -----------------------------------------
def hyperparams_box(model_name):
    if model_name == "K-Nearest Neighbors Regressor":
        return widgets.VBox([knn_k, knn_k_note])
    elif model_name == "Support Vector Regressor with Radial Basis Function kernel":
        return widgets.VBox([svr_c, svr_c_note, svr_eps, svr_eps_note])
    elif model_name == "Random Forest Regressor":
        return widgets.VBox([rf_n, rf_n_note, rf_depth, rf_depth_note])
    elif model_name == "Gradient Boosting Regressor":
        return widgets.VBox([gb_n, gb_n_note, gb_lr, gb_lr_note, gb_depth, gb_depth_note])
    elif model_name == "Artificial Neural Network (Multilayer Perceptron Regressor)":
        return widgets.VBox([
            ann_layers, ann_layers_note,
            ann_h1, ann_h2, ann_h3, ann_h4,
            ann_act, ann_act_note,
            ann_solver, ann_solver_note,
            ann_lr_init, ann_lr_init_note,
            ann_alpha, ann_alpha_note,
            ann_max_iter, ann_max_iter_note,
            ann_early, ann_early_note
        ])
    else:
        return widgets.VBox([widgets.HTML("<small>No tunable hyperparameters are exposed for Linear Regression.</small>")])

hp_container = widgets.VBox([hyperparams_box(model_dropdown.value)])

def on_model_change(change):
    if change["name"] == "value":
        hp_container.children = [hyperparams_box(change["new"])]
model_dropdown.observe(on_model_change)

def _hidden_layer_sizes():
    # Build tuple according to number of layers selected
    n = int(ann_layers.value)
    sizes = []
    if n >= 1: sizes.append(int(ann_h1.value))
    if n >= 2: sizes.append(int(ann_h2.value))
    if n >= 3: sizes.append(int(ann_h3.value))
    if n >= 4: sizes.append(int(ann_h4.value))
    return tuple(sizes)

def rebuild_pipeline(selected_model):
    if selected_model == "Linear Regression":
        pipe = Pipeline([("model", LinearRegression())])
    elif selected_model == "K-Nearest Neighbors Regressor":
        pipe = Pipeline([
            ("scaler", StandardScaler()),
            ("model", KNeighborsRegressor(n_neighbors=int(knn_k.value)))
        ])
    elif selected_model == "Support Vector Regressor with Radial Basis Function kernel":
        pipe = Pipeline([
            ("scaler", StandardScaler()),
            ("model", SVR(C=float(svr_c.value), epsilon=float(svr_eps.value), kernel="rbf"))
        ])
    elif selected_model == "Random Forest Regressor":
        depth = None if int(rf_depth.value) == 0 else int(rf_depth.value)
        pipe = Pipeline([
            ("model", RandomForestRegressor(
                n_estimators=int(rf_n.value),
                max_depth=depth,
                random_state=42,
                n_jobs=-1
            ))
        ])
    elif selected_model == "Gradient Boosting Regressor":
        pipe = Pipeline([
            ("model", GradientBoostingRegressor(
                n_estimators=int(gb_n.value),
                learning_rate=float(gb_lr.value),
                max_depth=int(gb_depth.value),
                random_state=42
            ))
        ])
    elif selected_model == "Artificial Neural Network (Multilayer Perceptron Regressor)":
        pipe = Pipeline([
            ("scaler", StandardScaler()),
            ("model", MLPRegressor(
                hidden_layer_sizes=_hidden_layer_sizes(),
                activation=str(ann_act.value),
                solver=str(ann_solver.value),
                learning_rate_init=float(ann_lr_init.value),
                alpha=float(ann_alpha.value),
                max_iter=int(ann_max_iter.value),
                early_stopping=bool(ann_early.value),
                validation_fraction=0.1,
                random_state=42
            ))
        ])
    else:
        raise ValueError(f"Unknown model: {selected_model}")
    return pipe

fitted_pipe = {"model": None, "name": None}

def _basic_plots_efficiency(oof_eff_true, oof_eff_pred, X_ref, pipe, feature_columns, conc_name="Coagulant concentration [mL/L]"):
    """
    Render the 4 basic plots:
      1) Observed vs Predicted Efficiency [%]
      2) Residuals vs Predicted (Efficiency)
      3) Histogram of Residuals (Efficiency)
      4) 1D Partial Dependence of Efficiency on Coagulant concentration
    """
    # 1) Parity plot (Efficiency)
    plt.figure(figsize=(5, 5))
    plt.scatter(oof_eff_true, oof_eff_pred, alpha=0.6)
    plt.plot([0, 100], [0, 100], linestyle='--')
    plt.xlim(0, 100)
    plt.ylim(0, 100)
    plt.xlabel("Observed Efficiency [%]")
    plt.ylabel("Predicted Efficiency [%]")
    plt.title("Observed vs. Predicted Efficiency [%]")
    plt.grid(True)
    plt.show()

    # 2) Residuals vs Predicted (Efficiency)
    res = oof_eff_true - oof_eff_pred
    plt.figure(figsize=(6, 4))
    plt.scatter(oof_eff_pred, res, alpha=0.6)
    plt.axhline(0.0, linestyle='--')
    plt.xlabel("Predicted Efficiency [%]")
    plt.ylabel("Residual (Observed - Predicted) [%]")
    plt.title("Residuals vs. Predicted (Efficiency)")
    plt.grid(True)
    plt.show()

    # 3) Histogram of Residuals (Efficiency)
    plt.figure(figsize=(6, 4))
    plt.hist(res, bins=30)
    plt.xlabel("Residual (Observed - Predicted) [%]")
    plt.ylabel("Count")
    plt.title("Histogram of Residuals (Efficiency)")
    plt.grid(True)
    plt.show()

    # 4) 1D Partial Dependence on Coagulant concentration (Efficiency)
    if conc_name in feature_columns:
        x_median = X_ref[feature_columns].median(numeric_only=True)
        # Build grid from 5th to 95th percentile of the concentration in X_ref
        conc_vals = X_ref[conc_name].dropna().values
        if len(conc_vals) > 0:
            low = float(np.percentile(conc_vals, 5))
            high = float(np.percentile(conc_vals, 95))
            grid = np.linspace(low, high, 50)
            rows = []
            for g in grid:
                row = x_median.copy()
                row[conc_name] = g
                # Round integer-only inputs to integer
                for int_col in ["Mixing speed [rpm]", "Global velocity gradient [s-1]", "Sedimentation time [min]"]:
                    if int_col in row.index:
                        row[int_col] = int(round(float(row[int_col])))
                rows.append(row.values.tolist())
            X_grid = pd.DataFrame(rows, columns=feature_columns)
            eff_grid = pipe.predict(X_grid)
            eff_grid = np.clip(eff_grid, 0.0, 100.0)

            plt.figure(figsize=(6, 4))
            plt.plot(grid, eff_grid)
            plt.xlabel("Coagulant concentration [mL/L]")
            plt.ylabel("Predicted Efficiency [%]")
            plt.title("Partial Dependence: Efficiency vs. Coagulant concentration")
            plt.grid(True)
            plt.show()

@output_area.capture(clear_output=True)
def on_train_clicked(_):
    selected = model_dropdown.value
    pipe = rebuild_pipeline(selected)

    # Evaluate with GroupKFold on turbidity metrics (train on efficiency) and get OOF predictions
    metrics = evaluate_with_groupkfold(pipe, X, Y_TURB, Y_EFF, T0_VEC, GROUPS, n_splits=5, return_oof=True)

    # Fit on full dataset to efficiency target
    pipe.fit(X, Y_EFF)
    fitted_pipe["model"] = pipe
    fitted_pipe["name"] = selected

    display(Markdown(f"### Training & Cross-Validation Results — **{selected}**"))
    print(f"Group 5-fold CV R² (final turbidity): {metrics['R2_mean']:.3f} ± {metrics['R2_std']:.3f}")
    print(f"Group 5-fold CV RMSE (final turbidity): {metrics['RMSE_mean']:.3f} ± {metrics['RMSE_std']:.3f} NTU")
    if np.isfinite(metrics["NRMSE_pct_mean"]):
        print(f"Group 5-fold CV Normalized RMSE:      {metrics['NRMSE_pct_mean']:.2f}% ± {metrics['NRMSE_pct_std']:.2f}% "
              f"(relative to dataset mean turbidity ≈ {metrics['y_mean']:.3f} NTU)")
    else:
        print("Group 5-fold CV Normalized RMSE:      N/A (dataset mean turbidity is zero or undefined)")
    print("\nModel is now fitted (efficiency target) on the full dataset and ready for prediction.")

    # ----- BASIC PLOTTING KIT (uses OOF efficiency predictions to avoid bias)
    oof = metrics.get("oof", None)
    if oof is not None:
        _basic_plots_efficiency(
            oof_eff_true=oof["eff_true"],
            oof_eff_pred=oof["eff_pred"],
            X_ref=X,  # use full X for medians and percentiles
            pipe=pipe,
            feature_columns=FEATURE_COLUMNS
        )
    else:
        print("OOF predictions not available for plotting.")

@output_area.capture(clear_output=False)
def on_predict_clicked(_):
    if fitted_pipe["model"] is None:
        print("Please train a model first (click 'Train and Evaluate (Group 5-fold Cross-Validation)')).")
        return

    # Collect user inputs (blk_* is a VBox [title, field, note]; field is children[1])
    ui_vec = {
        "Coagulant concentration [mL/L]": float(blk_conc.children[1].value),
        "Coagulation/flocculation time [min]": float(blk_ctime.children[1].value),
        "Mixing speed [rpm]": int(blk_mix.children[1].value),
        "Global velocity gradient [s-1]": int(blk_g.children[1].value),
        "Sedimentation time [min]": int(blk_sed.children[1].value),
        "Turbidity at time zero [NTU]": float(blk_t0.children[1].value),
    }

    # Reorder to match training feature order
    x_df = pd.DataFrame([[ui_vec[c] for c in FEATURE_COLUMNS]], columns=FEATURE_COLUMNS)

    # 1) Predict Efficiency [%] and clip to [0, 100]
    eff_pred = float(fitted_pipe["model"].predict(x_df)[0])
    eff_pred = float(np.clip(eff_pred, 0.0, 100.0))

    # 2) Compute final turbidity from user-provided T0
    T0_user = float(ui_vec["Turbidity at time zero [NTU]"])
    final_turb = T0_user * (1.0 - eff_pred / 100.0)

    display(Markdown(
        f"**Predicted Turbidity Removal Efficiency [%]:** `{eff_pred:.2f}%`  (Model: {fitted_pipe['name']})"
        f"<br>**Turbidity at time zero [NTU] (user-provided):** `{T0_user:.3f}`"
        f"<br>**Predicted final Turbidity [NTU]:** `{final_turb:.3f}`"
    ))

train_button.on_click(on_train_clicked)
predict_button.on_click(on_predict_clicked)

# -----------------------------------------
# 10) Sectioned UI (Header | Model | Inputs & Actions)
# -----------------------------------------
sec1_title = widgets.HTML("<h3 style='margin:0'>Turbidity Prediction Tool — Aloe vera Coagulant</h3>")
sec1_desc = widgets.HTML(
    "<p style='margin:8px 0 0 0'>This tool trains models to predict <b>Turbidity Removal Efficiency [%]</b> "
    "using initial turbidity derived from the dataset, guaranteeing physical bounds (0–100%). "
    "At prediction time, provide your <b>time-zero turbidity</b> to obtain both the efficiency and the "
    "corresponding <b>final turbidity [NTU]</b>.</p>"
    "<ul style='margin-top:8px'>"
    "<li><b>TRIOR</b> denotes triplicates conceptually; grouping keeps identical settings together in cross-validation.</li>"
    "<li>Sections: <i>Header</i>, <i>Model & Hyperparameters</i>, and <i>User Inputs & Actions</i>.</li>"
    "</ul>"
)
section_header = widgets.VBox([sec1_title, sec1_desc], layout=widgets.Layout(
    border='1px solid #ddd', padding='16px', margin='12px 0', width='100%'
))

sec2_title = widgets.HTML("<h4 style='margin:0 0 8px 0'>Model selection and hyperparameter configuration</h4>")
section_model = widgets.VBox([
    sec2_title,
    model_dropdown,
    widgets.HTML("<hr>"),
    widgets.HTML("<b>Hyperparameters (full labels with concise meanings)</b>"),
    hp_container
], layout=widgets.Layout(
    border='1px solid #ddd', padding='16px', margin='12px 0', width='100%',
    overflow_x='visible', overflow_y='visible'
))

sec3_title = widgets.HTML("<h4 style='margin:0 0 8px 0'>User inputs and actions</h4>")

# Responsive grid: no horizontal scroll; wraps to 1+ columns
inputs_grid = widgets.GridBox(
    children=[blk_t0, blk_conc, blk_ctime, blk_mix, blk_g, blk_sed],
    layout=widgets.Layout(
        grid_template_columns="repeat(auto-fit, minmax(214px, 1fr))",
        grid_gap="14px 24px",
        width='100%',
        max_width='100%',
        overflow_x='hidden',
        overflow_y='visible',
        justify_content='flex-start',
        align_items='flex-start'
    )
)

# Buttons container unchanged (buttons remain 578px wide)
buttons_box = widgets.VBox([
    train_button,
    widgets.HTML("&nbsp;"),
    predict_button
])

section_inputs = widgets.VBox([
    sec3_title,
    widgets.HTML("<b>Input parameters for prediction (full names)</b>"),
    inputs_grid,
    widgets.HTML("<hr>"),
    buttons_box,
    output_area
], layout=widgets.Layout(
    border='1px solid #ddd', padding='16px', margin='12px 0', width='100%',
    overflow_x='visible', overflow_y='visible'
))

ui = widgets.VBox([section_header, section_model, section_inputs])
display(ui)



Dataset.xlsx not found in default locations. Please upload it now.


Saving 99_Dataset.xlsx to 99_Dataset.xlsx
Loaded dataset from uploaded file: 99_Dataset.xlsx
